# Web Application Vulnerabilities - Hands-On Lab

**Part of HackLearn Pro - Module #19**

This interactive notebook provides comprehensive hands-on exercises for understanding critical web application vulnerabilities including CSRF, SSRF, XXE, insecure deserialization, path traversal, and file upload attacks.

**Learning Objectives:**
- Understand and exploit CSRF vulnerabilities
- Test for SSRF in cloud environments
- Craft XXE payloads for file disclosure
- Exploit insecure deserialization
- Bypass file upload restrictions
- Implement comprehensive security headers
- Configure Web Application Firewalls

**WARNING:** These techniques are for educational and authorized testing purposes only. Unauthorized testing is illegal under CFAA (18 USC § 1030) and similar laws worldwide. Always obtain written permission before testing.

## Lab 1: CSRF Token Generation and Validation

Implement secure CSRF protection using cryptographically strong tokens and validation.

In [ ]:
import secrets
import hashlib
import hmac
import time
from typing import Dict, Optional

class CSRFProtection:
    """Secure CSRF token generation and validation."""
    
    def __init__(self, secret_key: str):
        self.secret_key = secret_key.encode()
        self.tokens: Dict[str, float] = {}  # token -> timestamp
    
    def generate_token(self, session_id: str) -> str:
        """Generate cryptographically secure CSRF token."""
        # Random component
        random_bytes = secrets.token_bytes(32)
        
        # Timestamp
        timestamp = str(time.time()).encode()
        
        # Session binding
        session_bytes = session_id.encode()
        
        # Create HMAC signature
        message = random_bytes + timestamp + session_bytes
        signature = hmac.new(self.secret_key, message, hashlib.sha256).digest()
        
        # Combine and encode
        token_bytes = random_bytes + signature
        token = token_bytes.hex()
        
        # Store with timestamp for expiry
        self.tokens[token] = time.time()
        
        return token
    
    def validate_token(self, token: str, session_id: str, max_age: int = 3600) -> bool:
        """Validate CSRF token."""
        # Check if token exists
        if token not in self.tokens:
            print("Token not found in storage")
            return False
        
        # Check expiry
        token_time = self.tokens[token]
        if time.time() - token_time > max_age:
            print(f"Token expired (age: {time.time() - token_time:.1f}s)")
            del self.tokens[token]
            return False
        
        try:
            # Decode token
            token_bytes = bytes.fromhex(token)
            random_bytes = token_bytes[:32]
            provided_signature = token_bytes[32:]
            
            # Reconstruct message
            timestamp = str(token_time).encode()
            session_bytes = session_id.encode()
            message = random_bytes + timestamp + session_bytes
            
            # Compute expected signature
            expected_signature = hmac.new(
                self.secret_key,
                message,
                hashlib.sha256
            ).digest()
            
            # Constant-time comparison
            if not hmac.compare_digest(provided_signature, expected_signature):
                print("Signature verification failed")
                return False
            
            # One-time use: remove token after validation
            del self.tokens[token]
            
            return True
        except Exception as e:
            print(f"Validation error: {e}")
            return False

# Test CSRF protection
print("CSRF Token Security Testing")
print("=" * 60)

csrf = CSRFProtection(secret_key='your-super-secret-key-here-change-me')
session_id = 'user_session_12345'

print("\n1. Generate CSRF token:")
token = csrf.generate_token(session_id)
print(f"Token: {token[:40]}...")
print(f"Token length: {len(token)} characters")

print("\n2. Validate correct token:")
is_valid = csrf.validate_token(token, session_id)
print(f"Valid: {is_valid}")

print("\n3. Try to reuse token (should fail - one-time use):")
is_valid = csrf.validate_token(token, session_id)
print(f"Valid: {is_valid}")

print("\n4. Tamper with token (should fail):")
token2 = csrf.generate_token(session_id)
tampered_token = token2[:-4] + 'abcd'  # Change last 2 bytes
is_valid = csrf.validate_token(tampered_token, session_id)
print(f"Valid: {is_valid}")

print("\n5. Use token with wrong session (should fail):")
token3 = csrf.generate_token(session_id)
is_valid = csrf.validate_token(token3, 'wrong_session_id')
print(f"Valid: {is_valid}")

print("\n6. Test token expiry:")
token4 = csrf.generate_token(session_id)
csrf.tokens[token4] = time.time() - 3700  # Make it 1+ hour old
is_valid = csrf.validate_token(token4, session_id, max_age=3600)
print(f"Valid: {is_valid}")

print("\n" + "=" * 60)
print("Key Security Properties:")
print("- Cryptographically random (32 bytes)")
print("- HMAC signature prevents tampering")
print("- Session binding prevents token stealing")
print("- Time-based expiry (default 1 hour)")
print("- One-time use prevents replay attacks")
print("- Constant-time comparison prevents timing attacks")

## Lab 2: SSRF Detection and Exploitation

Test for Server-Side Request Forgery vulnerabilities, especially in cloud environments.

In [ ]:
import socket
import ipaddress
from urllib.parse import urlparse
from typing import List, Tuple

class SSRFTester:
    """SSRF vulnerability detection and testing."""
    
    # Common cloud metadata endpoints
    METADATA_ENDPOINTS = [
        'http://169.254.169.254/latest/meta-data/',  # AWS
        'http://metadata.google.internal/computeMetadata/v1/',  # GCP
        'http://169.254.169.254/metadata/instance?api-version=2021-02-01',  # Azure
    ]
    
    # Dangerous internal IPs/ranges
    INTERNAL_RANGES = [
        '127.0.0.0/8',      # Localhost
        '10.0.0.0/8',       # Private A
        '172.16.0.0/12',    # Private B
        '192.168.0.0/16',   # Private C
        '169.254.0.0/16',   # Link-local (AWS metadata)
        '::1/128',          # IPv6 localhost
        'fc00::/7',         # IPv6 private
    ]
    
    def is_internal_ip(self, hostname: str) -> Tuple[bool, str]:
        """Check if hostname resolves to internal IP."""
        try:
            # Resolve hostname
            ip_str = socket.gethostbyname(hostname)
            ip = ipaddress.ip_address(ip_str)
            
            # Check against internal ranges
            for range_str in self.INTERNAL_RANGES:
                network = ipaddress.ip_network(range_str)
                if ip in network:
                    return True, f"{hostname} -> {ip_str} (in {range_str})"
            
            return False, f"{hostname} -> {ip_str} (external)"
        except Exception as e:
            return False, f"Error resolving {hostname}: {e}"
    
    def test_url(self, url: str) -> dict:
        """Test URL for SSRF indicators."""
        result = {
            'url': url,
            'is_ssrf_risk': False,
            'risks': []
        }
        
        try:
            parsed = urlparse(url)
            
            # Check scheme
            if parsed.scheme not in ['http', 'https']:
                result['risks'].append(f"Dangerous scheme: {parsed.scheme}")
                result['is_ssrf_risk'] = True
            
            # Check for metadata endpoints
            for metadata_url in self.METADATA_ENDPOINTS:
                if url.startswith(metadata_url):
                    result['risks'].append(f"Cloud metadata endpoint detected")
                    result['is_ssrf_risk'] = True
            
            # Check hostname resolution
            if parsed.hostname:
                is_internal, msg = self.is_internal_ip(parsed.hostname)
                if is_internal:
                    result['risks'].append(msg)
                    result['is_ssrf_risk'] = True
            
            # Check for localhost variations
            localhost_variants = [
                'localhost', '127.0.0.1', '0.0.0.0', '0177.0.0.01',  # Octal
                '2130706433',  # Decimal
                '0x7f.0x0.0x0.0x1',  # Hex
            ]
            if parsed.hostname in localhost_variants:
                result['risks'].append(f"Localhost variant: {parsed.hostname}")
                result['is_ssrf_risk'] = True
            
        except Exception as e:
            result['error'] = str(e)
        
        return result

# Test SSRF detection
print("SSRF Vulnerability Testing")
print("=" * 80)

tester = SSRFTester()

# Test URLs
test_urls = [
    'https://api.example.com/data',  # Safe
    'http://169.254.169.254/latest/meta-data/iam/security-credentials/',  # AWS metadata
    'http://localhost:8080/admin',  # Localhost
    'http://127.0.0.1/admin',  # Localhost IP
    'http://192.168.1.1/config',  # Private network
    'file:///etc/passwd',  # File protocol
    'http://metadata.google.internal/computeMetadata/v1/',  # GCP metadata
    'http://10.0.0.5:9200/_cluster/health',  # Internal Elasticsearch
]

for url in test_urls:
    print(f"\nTesting: {url}")
    result = tester.test_url(url)
    
    if result['is_ssrf_risk']:
        print("STATUS: SSRF RISK DETECTED")
        print("Risks:")
        for risk in result['risks']:
            print(f"  - {risk}")
    else:
        print("STATUS: Safe (no SSRF indicators)")
    
    if 'error' in result:
        print(f"Error: {result['error']}")

print("\n" + "=" * 80)
print("\nCommon SSRF Payloads to Test:")
print("-" * 80)
payloads = [
    'http://169.254.169.254/latest/meta-data/',
    'http://127.0.0.1:22',  # Check SSH
    'http://127.0.0.1:3306',  # Check MySQL
    'http://127.0.0.1:6379',  # Check Redis
    'http://[::1]/admin',  # IPv6 localhost
    'http://127.1/admin',  # Short form localhost
    'http://0x7f000001/admin',  # Hex localhost
    'file:///etc/passwd',  # File protocol
    'dict://127.0.0.1:6379/info',  # DICT protocol
    'gopher://127.0.0.1:9000/_',  # Gopher protocol
]

for i, payload in enumerate(payloads, 1):
    print(f"{i:2}. {payload}")

print("\nDefense Strategies:")
print("1. Whitelist allowed domains/IPs only")
print("2. Block access to private IP ranges (RFC1918)")
print("3. Block access to localhost (127.0.0.0/8, ::1)")
print("4. Block cloud metadata endpoints (169.254.169.254)")
print("5. Use allow-lists for protocols (http/https only)")
print("6. Disable HTTP redirects or validate redirect targets")
print("7. Implement network-level controls (VPC, security groups)")
print("8. Use IMDSv2 (AWS) with session tokens")

## Lab 3: XXE (XML External Entity) Exploitation

Demonstrate XXE attacks for file disclosure and SSRF, then implement secure XML parsing.

In [ ]:
import xml.etree.ElementTree as ET
from defusedxml.ElementTree import parse as safe_parse
from io import StringIO

# VULNERABLE: Standard XML parsing (DO NOT USE IN PRODUCTION)
print("XXE Vulnerability Demonstration")
print("=" * 80)

print("\n1. VULNERABLE CODE: Standard XML Parser")
print("-" * 80)

# Example 1: File disclosure via XXE
xxe_payload_file = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ELEMENT foo ANY>
  <!ENTITY xxe SYSTEM "file:///etc/passwd">
]>
<data>
  <username>admin</username>
  <file>&xxe;</file>
</data>'''

print("Malicious XML Payload (File Disclosure):")
print(xxe_payload_file)
print("\nAttempting to parse (this may fail with error, which is good):")

try:
    # Vulnerable parsing - DO NOT USE
    tree = ET.parse(StringIO(xxe_payload_file))
    root = tree.getroot()
    
    for child in root:
        print(f"{child.tag}: {child.text}")
        if child.tag == 'file' and child.text:
            print("\nALERT: XXE vulnerability present! File contents leaked:")
            print(child.text[:200] + '...' if len(child.text) > 200 else child.text)
except Exception as e:
    print(f"Parsing failed (expected): {type(e).__name__}")
    print("Note: Some XML parsers reject external entities by default")

# Example 2: SSRF via XXE
print("\n" + "=" * 80)
print("\n2. XXE for SSRF (Server-Side Request Forgery)")
print("-" * 80)

xxe_payload_ssrf = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ELEMENT foo ANY>
  <!ENTITY xxe SYSTEM "http://169.254.169.254/latest/meta-data/iam/security-credentials/">
]>
<data>
  <query>&xxe;</query>
</data>'''

print("Malicious XML Payload (SSRF to AWS Metadata):")
print(xxe_payload_ssrf)

# Example 3: Billion Laughs Attack (XML Bomb)
print("\n" + "=" * 80)
print("\n3. Billion Laughs Attack (Denial of Service)")
print("-" * 80)

billion_laughs = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE lolz [
  <!ENTITY lol "lol">
  <!ENTITY lol2 "&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;&lol;">
  <!ENTITY lol3 "&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;&lol2;">
  <!ENTITY lol4 "&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;&lol3;">
]>
<data>&lol4;</data>'''

print("XML Bomb Payload (causes exponential entity expansion):")
print(billion_laughs)
print("\nThis expands to gigabytes of data, causing DoS")

# SECURE: Using defusedxml
print("\n" + "=" * 80)
print("\n4. SECURE CODE: Using defusedxml Library")
print("-" * 80)

safe_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<data>
  <username>admin</username>
  <action>login</action>
</data>'''

print("Safe XML (no external entities):")
print(safe_xml)
print("\nParsing with defusedxml:")

try:
    tree = safe_parse(StringIO(safe_xml))
    root = tree.getroot()
    
    print("SUCCESS: Parsed safely")
    for child in root:
        print(f"  {child.tag}: {child.text}")
except Exception as e:
    print(f"Error: {e}")

print("\nTrying to parse XXE payload with defusedxml:")
try:
    tree = safe_parse(StringIO(xxe_payload_file))
    print("ALERT: XXE payload was parsed (should not happen)")
except Exception as e:
    print(f"SUCCESS: XXE blocked - {type(e).__name__}")
    print("defusedxml successfully prevented XXE attack")

# Secure configuration guide
print("\n" + "=" * 80)
print("\nSecure XML Parsing Configuration")
print("-" * 80)
print("""
Python (lxml):
  parser = etree.XMLParser(
      resolve_entities=False,
      no_network=True,
      dtd_validation=False,
      load_dtd=False
  )

Java:
  DocumentBuilderFactory dbf = DocumentBuilderFactory.newInstance();
  dbf.setFeature("http://apache.org/xml/features/disallow-doctype-decl", true);
  dbf.setFeature("http://xml.org/sax/features/external-general-entities", false);
  dbf.setFeature("http://xml.org/sax/features/external-parameter-entities", false);
  dbf.setXIncludeAware(false);
  dbf.setExpandEntityReferences(false);

PHP:
  libxml_disable_entity_loader(true);
  $dom = new DOMDocument();
  $dom->loadXML($xml, LIBXML_NOENT | LIBXML_DTDLOAD | LIBXML_DTDATTR);

.NET:
  XmlReaderSettings settings = new XmlReaderSettings();
  settings.DtdProcessing = DtdProcessing.Prohibit;
  settings.XmlResolver = null;
""")

print("\nKey Defenses:")
print("1. Use secure XML parsers (defusedxml, lxml with safe config)")
print("2. Disable DTD processing completely")
print("3. Disable external entity resolution")
print("4. Use JSON instead of XML when possible")
print("5. Implement input validation and size limits")
print("6. Keep XML libraries updated")

## Lab 4: File Upload Security Testing

Test file upload restrictions and implement secure file handling.

In [ ]:
import os
import magic  # python-magic library
import hashlib
from pathlib import Path

class SecureFileUpload:
    """Secure file upload validation and handling."""
    
    # Allowed MIME types (whitelist approach)
    ALLOWED_TYPES = {
        'image/jpeg': ['.jpg', '.jpeg'],
        'image/png': ['.png'],
        'image/gif': ['.gif'],
        'application/pdf': ['.pdf'],
        'text/plain': ['.txt'],
    }
    
    # Dangerous extensions to explicitly block
    BLOCKED_EXTENSIONS = [
        '.php', '.php3', '.php4', '.php5', '.phtml',
        '.exe', '.dll', '.bat', '.cmd', '.com',
        '.sh', '.bash', '.zsh',
        '.jsp', '.jspx', '.asp', '.aspx',
        '.py', '.rb', '.pl', '.cgi',
        '.jar', '.war',
    ]
    
    MAX_FILE_SIZE = 10 * 1024 * 1024  # 10 MB
    
    def __init__(self, upload_dir: str):
        self.upload_dir = Path(upload_dir)
        self.upload_dir.mkdir(parents=True, exist_ok=True)
    
    def validate_file(self, filename: str, content: bytes) -> dict:
        """Comprehensive file validation."""
        result = {
            'valid': False,
            'errors': [],
            'warnings': []
        }
        
        # 1. Check file size
        if len(content) > self.MAX_FILE_SIZE:
            result['errors'].append(
                f"File too large: {len(content)} bytes (max: {self.MAX_FILE_SIZE})"
            )
            return result
        
        if len(content) == 0:
            result['errors'].append("Empty file")
            return result
        
        # 2. Validate filename
        if not filename or filename.startswith('.'):
            result['errors'].append("Invalid filename")
            return result
        
        # Check for path traversal
        if '..' in filename or '/' in filename or '\\' in filename:
            result['errors'].append("Path traversal attempt detected")
            return result
        
        # 3. Check extension
        file_ext = Path(filename).suffix.lower()
        
        if file_ext in self.BLOCKED_EXTENSIONS:
            result['errors'].append(f"Blocked extension: {file_ext}")
            return result
        
        # 4. Detect actual MIME type (magic bytes)
        mime_type = magic.from_buffer(content, mime=True)
        
        if mime_type not in self.ALLOWED_TYPES:
            result['errors'].append(f"Disallowed MIME type: {mime_type}")
            return result
        
        # 5. Verify extension matches MIME type
        expected_extensions = self.ALLOWED_TYPES[mime_type]
        if file_ext not in expected_extensions:
            result['errors'].append(
                f"Extension mismatch: {file_ext} doesn't match MIME type {mime_type}"
            )
            return result
        
        # 6. Additional checks for images
        if mime_type.startswith('image/'):
            # Check for embedded PHP in images
            if b'<?php' in content or b'<?=' in content:
                result['errors'].append("PHP code detected in image file")
                return result
        
        result['valid'] = True
        result['mime_type'] = mime_type
        result['extension'] = file_ext
        
        return result
    
    def generate_safe_filename(self, original_filename: str) -> str:
        """Generate cryptographically secure filename."""
        # Get extension
        ext = Path(original_filename).suffix.lower()
        
        # Generate random filename
        random_hash = hashlib.sha256(os.urandom(32)).hexdigest()[:16]
        
        return f"{random_hash}{ext}"
    
    def save_file(self, filename: str, content: bytes) -> dict:
        """Validate and save uploaded file securely."""
        # Validate
        validation = self.validate_file(filename, content)
        
        if not validation['valid']:
            return validation
        
        # Generate safe filename
        safe_filename = self.generate_safe_filename(filename)
        file_path = self.upload_dir / safe_filename
        
        # Ensure path is within upload directory (prevent traversal)
        try:
            file_path.resolve().relative_to(self.upload_dir.resolve())
        except ValueError:
            validation['valid'] = False
            validation['errors'].append("Path traversal detected")
            return validation
        
        # Save file
        with open(file_path, 'wb') as f:
            f.write(content)
        
        # Set restrictive permissions (owner read/write only)
        os.chmod(file_path, 0o600)
        
        validation['saved_path'] = str(file_path)
        validation['saved_filename'] = safe_filename
        
        return validation

# Test file upload security
print("Secure File Upload Testing")
print("=" * 80)

uploader = SecureFileUpload('/tmp/test_uploads')

# Test cases
test_files = [
    {
        'name': 'legitimate_image.jpg',
        'content': b'\xff\xd8\xff\xe0\x00\x10JFIF',  # JPEG magic bytes
        'should_pass': True
    },
    {
        'name': 'fake_image.jpg',
        'content': b'<?php system($_GET["cmd"]); ?>',  # PHP web shell
        'should_pass': False
    },
    {
        'name': 'shell.php',
        'content': b'<?php system("ls"); ?>',
        'should_pass': False
    },
    {
        'name': 'traversal.jpg',
        'content': b'\x89PNG\r\n',  # PNG magic bytes but wrong extension
        'should_pass': False
    },
    {
        'name': '../../../etc/passwd',
        'content': b'malicious',
        'should_pass': False
    },
]

for i, test_file in enumerate(test_files, 1):
    print(f"\nTest {i}: {test_file['name']}")
    print("-" * 60)
    
    result = uploader.validate_file(test_file['name'], test_file['content'])
    
    print(f"Expected: {'PASS' if test_file['should_pass'] else 'FAIL'}")
    print(f"Actual: {'PASS' if result['valid'] else 'FAIL'}")
    
    if result['errors']:
        print("Errors:")
        for error in result['errors']:
            print(f"  - {error}")
    
    if result['warnings']:
        print("Warnings:")
        for warning in result['warnings']:
            print(f"  - {warning}")
    
    # Check if result matches expectation
    if result['valid'] == test_file['should_pass']:
        print("Result: CORRECT")
    else:
        print("Result: INCORRECT (validation logic may need adjustment)")

print("\n" + "=" * 80)
print("\nFile Upload Security Checklist:")
print("-" * 80)
print("""
1. Validate file size (prevent DoS)
2. Validate MIME type using magic bytes (not extension)
3. Whitelist allowed file types only
4. Blacklist dangerous extensions (.php, .exe, etc.)
5. Check for path traversal in filename
6. Rename files to random names (prevent overwriting)
7. Store files outside web root
8. Set restrictive file permissions (0600 or 0400)
9. Scan for malware/viruses (ClamAV)
10. Implement rate limiting per user/IP
11. Strip EXIF metadata from images
12. Use separate domain for serving user content
13. Implement Content-Security-Policy headers
14. Never execute uploaded files
15. Monitor upload directory for suspicious files
""")

## Lab 5: Security Headers Implementation

Configure comprehensive security headers for web application protection.

In [ ]:
from typing import Dict

class SecurityHeaders:
    """Generate and validate security headers."""
    
    @staticmethod
    def get_recommended_headers() -> Dict[str, str]:
        """Get recommended security headers configuration."""
        return {
            # Prevent clickjacking attacks
            'X-Frame-Options': 'DENY',
            
            # Prevent MIME type sniffing
            'X-Content-Type-Options': 'nosniff',
            
            # Enable XSS filter (legacy, but still useful)
            'X-XSS-Protection': '1; mode=block',
            
            # Enforce HTTPS
            'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload',
            
            # Content Security Policy (strict)
            'Content-Security-Policy': (
                "default-src 'self'; "
                "script-src 'self'; "
                "style-src 'self' 'unsafe-inline'; "
                "img-src 'self' data: https:; "
                "font-src 'self'; "
                "connect-src 'self'; "
                "frame-ancestors 'none'; "
                "base-uri 'self'; "
                "form-action 'self'"
            ),
            
            # Referrer policy
            'Referrer-Policy': 'strict-origin-when-cross-origin',
            
            # Permissions policy (feature policy)
            'Permissions-Policy': (
                "geolocation=(), "
                "camera=(), "
                "microphone=(), "
                "payment=(), "
                "usb=(), "
                "magnetometer=(), "
                "accelerometer=(), "
                "gyroscope=()"
            ),
            
            # Cross-Origin policies
            'Cross-Origin-Embedder-Policy': 'require-corp',
            'Cross-Origin-Opener-Policy': 'same-origin',
            'Cross-Origin-Resource-Policy': 'same-origin',
        }
    
    @staticmethod
    def validate_headers(headers: Dict[str, str]) -> Dict:
        """Validate security headers and provide recommendations."""
        recommended = SecurityHeaders.get_recommended_headers()
        
        result = {
            'score': 0,
            'max_score': len(recommended),
            'present': [],
            'missing': [],
            'warnings': []
        }
        
        for header, recommended_value in recommended.items():
            if header in headers:
                result['present'].append(header)
                result['score'] += 1
                
                # Check for weak configurations
                actual_value = headers[header]
                
                if header == 'X-Frame-Options':
                    if actual_value.upper() not in ['DENY', 'SAMEORIGIN']:
                        result['warnings'].append(
                            f"{header}: '{actual_value}' is weak, use 'DENY' or 'SAMEORIGIN'"
                        )
                
                elif header == 'Strict-Transport-Security':
                    if 'max-age' not in actual_value:
                        result['warnings'].append(
                            f"{header}: Missing 'max-age' directive"
                        )
                    elif 'max-age=31536000' not in actual_value:
                        result['warnings'].append(
                            f"{header}: Recommended max-age is 31536000 (1 year)"
                        )
            else:
                result['missing'].append(header)
        
        result['grade'] = SecurityHeaders._calculate_grade(result['score'], result['max_score'])
        
        return result
    
    @staticmethod
    def _calculate_grade(score: int, max_score: int) -> str:
        """Calculate security grade."""
        percentage = (score / max_score) * 100
        
        if percentage >= 90:
            return 'A'
        elif percentage >= 80:
            return 'B'
        elif percentage >= 70:
            return 'C'
        elif percentage >= 60:
            return 'D'
        else:
            return 'F'

# Test security headers
print("Security Headers Analysis")
print("=" * 80)

# Example 1: Well-configured site
print("\n1. Well-Configured Site")
print("-" * 80)

good_headers = {
    'X-Frame-Options': 'DENY',
    'X-Content-Type-Options': 'nosniff',
    'X-XSS-Protection': '1; mode=block',
    'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload',
    'Content-Security-Policy': "default-src 'self'",
    'Referrer-Policy': 'strict-origin-when-cross-origin',
    'Permissions-Policy': 'geolocation=(), camera=()',
    'Cross-Origin-Embedder-Policy': 'require-corp',
    'Cross-Origin-Opener-Policy': 'same-origin',
    'Cross-Origin-Resource-Policy': 'same-origin',
}

result = SecurityHeaders.validate_headers(good_headers)

print(f"Score: {result['score']}/{result['max_score']}")
print(f"Grade: {result['grade']}")
print(f"\nPresent headers: {len(result['present'])}")
print(f"Missing headers: {len(result['missing'])}")

if result['missing']:
    print("\nMissing:")
    for header in result['missing']:
        print(f"  - {header}")

if result['warnings']:
    print("\nWarnings:")
    for warning in result['warnings']:
        print(f"  - {warning}")

# Example 2: Poorly configured site
print("\n" + "=" * 80)
print("\n2. Poorly-Configured Site")
print("-" * 80)

bad_headers = {
    'X-Frame-Options': 'ALLOWALL',  # Invalid value
    'Strict-Transport-Security': 'max-age=300',  # Too short
}

result = SecurityHeaders.validate_headers(bad_headers)

print(f"Score: {result['score']}/{result['max_score']}")
print(f"Grade: {result['grade']}")
print(f"\nPresent headers: {len(result['present'])}")
print(f"Missing headers: {len(result['missing'])}")

if result['missing']:
    print("\nMissing (critical):")
    for header in result['missing'][:5]:  # Show first 5
        print(f"  - {header}")
    if len(result['missing']) > 5:
        print(f"  ... and {len(result['missing']) - 5} more")

if result['warnings']:
    print("\nWarnings:")
    for warning in result['warnings']:
        print(f"  - {warning}")

# Show recommended configuration
print("\n" + "=" * 80)
print("\nRecommended Security Headers Configuration")
print("-" * 80)

recommended = SecurityHeaders.get_recommended_headers()
for header, value in recommended.items():
    print(f"\n{header}:")
    print(f"  {value}")

print("\n" + "=" * 80)
print("\nImplementation Examples:")
print("-" * 80)
print("""
Express.js / Node.js:
  const helmet = require('helmet');
  app.use(helmet());

Flask / Python:
  from flask_talisman import Talisman
  Talisman(app)

Nginx:
  add_header X-Frame-Options "DENY" always;
  add_header X-Content-Type-Options "nosniff" always;
  add_header X-XSS-Protection "1; mode=block" always;
  add_header Strict-Transport-Security "max-age=31536000; includeSubDomains" always;

Apache:
  Header always set X-Frame-Options "DENY"
  Header always set X-Content-Type-Options "nosniff"
  Header always set X-XSS-Protection "1; mode=block"
""")

## Summary and Key Takeaways

**Labs Completed:**
1. CSRF token generation and validation with HMAC signatures
2. SSRF detection for cloud metadata endpoints and internal networks
3. XXE exploitation and secure XML parsing
4. Secure file upload validation with MIME type verification
5. Comprehensive security headers configuration

**Defense-in-Depth Strategies:**
- **CSRF:** Synchronizer tokens, SameSite cookies, custom headers
- **SSRF:** URL whitelisting, internal IP blocking, metadata protection
- **XXE:** Disable external entities, use safe parsers, prefer JSON
- **File Upload:** Magic byte validation, random filenames, isolation
- **Headers:** CSP, HSTS, frame options, MIME type enforcement

**Critical Security Principles:**
1. **Never trust user input** - Validate everything
2. **Whitelist over blacklist** - Define what IS allowed
3. **Defense in depth** - Multiple layers of security
4. **Least privilege** - Minimal necessary permissions
5. **Fail securely** - Errors should not expose information
6. **Security headers** - Browser-level protection
7. **Regular updates** - Patch known vulnerabilities
8. **Security testing** - Continuous validation

**Next Steps:**
- Return to Module #19 in HackLearn Pro
- Complete the challenge quiz
- Explore the Tools and References tabs
- Practice on authorized platforms (DVWA, WebGoat, HackTheBox)
- Stay updated with OWASP Top 10

**Remember:** These vulnerabilities are present in millions of real-world applications. Understanding them deeply is essential for building secure web applications and protecting user data.